In [1]:
import cv2
import ternausnet.models
import numpy as np 
import torch
from PIL import Image
import numpy as np
model = ternausnet.models.UNet11(pretrained=False)
model.load_state_dict(torch.load('models2/model-from_scratch_sentinel_bs40_ne20_lr0.0001-9.pth'))
model = model.to(device='cuda:0')
# model.eval()

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [ ]:
im_path = 'bulgaria_iskar_s2cloudless_2016.jpg'
I = np.asarray(Image.open(im_path))


def inference(model, I, device='cuda:0', batch_size=40, threshold=0.5):
    I = np.asarray(Image.open(im_path))

    
    L1, L2 = I.shape[:2]
    d1 = 300
    d2 = 150

    crops = []

    for k, K in zip(list(range(0, L1 - d2, d2)), list(range(d1, L1 - d2, d2)) + [L1]):
        for p, P in zip(list(range(0, L2 - d2, d2)), list(range(d1, L2 - d2, d2)) + [L1]):
            crops.append(I[k:K, p:P])

    batch = np.stack(list(map(lambda x: cv2.resize(x, (224, 224), interpolation=cv2.INTER_AREA), crops)))
    
    batch = ((batch / 255) - np.array([0.485, 0.456, 0.406])[None, None, None, :]) / np.array([0.229, 0.224, 0.225])[None, None, None, :]
    
    batch = torch.tensor(batch, dtype=torch.float).to(device=device)
    
    batch = batch.transpose(2,3).transpose(1,2)
    
    res = np.concatenate(list(map(lambda x: (model(x) > threshold).detach().cpu().numpy().astype(np.uint8) * 255, torch.split(batch, batch_size))))
    
    C = np.zeros_like(I[:, :, 0])
    II = np.zeros_like(I[:, :, 0])
    
    i = 0
    for k, K in zip(list(range(0, L1 - d2, d2)), list(range(d1, L1 - d2, d2)) + [L1]):
        for p, P in zip(list(range(0, L2 - d2, d2)), list(range(d1, L2 - d2, d2)) + [L1]):
            res_ = cv2.resize(res[i, 0], crops[i].shape[:2][::-1], interpolation=cv2.INTER_LINEAR)
            II[k:K, p:P] += res_
            C[k:K, p:P] += 1
            i += 1
    II =  II / C
    
    return II

II = inference(model, I)

In [ ]:
%%prun
II = inference(model, I)

In [ ]:
%pylab inline

In [ ]:
III = I.copy()
III[:, :, 2] =cv2.morphologyEx(II, cv2.MORPH_GRADIENT, np.ones((5,5), dtype=np.uint8))

In [ ]:
plt.imshow(I)

In [ ]:
plt.imshow(II)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(III)